In [8]:
#!pip install delta-spark

In [9]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

sc = spark.sparkContext
sc

<SparkContext master=spark://spark-master:7077 appName=MyApp>

In [10]:
data = spark.range(0, 5)
data.write.format("delta").mode("overwrite").save("/data/delta/delta-table-qs")

In [11]:
df = spark.read.format("delta").load("/data/delta/delta-table-qs")
df.show()

+---+
| id|
+---+
|  2|
|  3|
|  4|
|  0|
|  1|
+---+



In [12]:
data = spark.range(5, 10)
data.write.format("delta").mode("overwrite").save("/data/delta/delta-table-qs")

In [13]:
from delta.tables import DeltaTable

delta_table = DeltaTable.forPath(spark, "/data/delta/delta-table-qs")

delta_table.history().select("version", "timestamp", "operation", "operationParameters").show(10, False)

+-------+-----------------------+---------+--------------------------------------+
|version|timestamp              |operation|operationParameters                   |
+-------+-----------------------+---------+--------------------------------------+
|17     |2024-03-25 22:45:22.009|WRITE    |{mode -> Overwrite, partitionBy -> []}|
|16     |2024-03-25 22:45:12.058|WRITE    |{mode -> Overwrite, partitionBy -> []}|
|15     |2024-03-25 22:34:50.564|WRITE    |{mode -> Overwrite, partitionBy -> []}|
|14     |2024-03-25 22:34:41.697|WRITE    |{mode -> Overwrite, partitionBy -> []}|
|13     |2024-03-25 01:25:36.076|WRITE    |{mode -> Overwrite, partitionBy -> []}|
|12     |2024-03-25 01:25:27.895|WRITE    |{mode -> Overwrite, partitionBy -> []}|
|11     |2024-03-25 01:22:15.792|WRITE    |{mode -> Overwrite, partitionBy -> []}|
|10     |2024-03-25 01:22:09.105|WRITE    |{mode -> Overwrite, partitionBy -> []}|
|9      |2024-03-25 01:09:40.499|WRITE    |{mode -> Overwrite, partitionBy -> []}|
|8  

In [14]:
spark.sparkContext.stop()